In [1]:
from tm2py.acceptance.acceptance import Acceptance
from tm2py.acceptance.canonical import Canonical
from tm2py.acceptance.simulated import Simulated
from tm2py.acceptance.observed import Observed

from pathlib import Path
import itertools
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import openmatrix as omx

In [2]:
scenario_config_file = "../examples/scenario_config.toml"
model_config_file = "../examples/model_config.toml"
observed_config_file = "../examples/observed_data.toml"
canonical_crosswalk_file = "../examples/canonical_crosswalk.toml"

output_folder_root = "/Users/wsp/Documents/GitHub/tm2py/examples/temp_acceptance/"

In [3]:
c = Canonical(canonical_crosswalk_file)

In [ ]:
s = Simulated(canonical=c, scenario_file=scenario_config_file, model_file=model_config_file)

In [ ]:
o = Observed(canonical=c, observed_file=observed_config_file)

In [ ]:
a = Acceptance(canonical=c, simulated=s, observed=o, output_file_root=output_folder_root)

In [ ]:
a.make_acceptance(make_transit=False, make_roadway=False, make_other=False)

In [ ]:
# assess successful and failed matches
self = a
df = self.c.standard_transit_to_survey_df.copy()

df["survey_agency"] = df["survey_agency"].map(self.c.canonical_agency_names_dict)
df = df[~df["survey_agency"].isin(self.c.rail_operators_vector)].copy()

df = c.aggregate_line_names_across_time_of_day(df.copy(), "standard_line_name")

survey_df = df[["survey_route", "survey_agency", "survey_tech", "canonical_operator", "daily_line_name"]].drop_duplicates().reset_index(drop=True).copy()

survey_df.head()


In [ ]:
sim_df = self.s.simulated_boardings_df[["daily_line_name", "total_boarding"]].copy()
sim_df.head()

In [ ]:
output_df = pd.merge(
    sim_df,
    survey_df,
    how = "left",
    on = "daily_line_name",
)

output_df.head()

In [ ]:
routes_df = self.s.standard_transit_routes_df.copy()
routes_df = routes_df[["agency_id", "route_short_name", "route_long_name", "route_id"]].copy()
routes_df.head()

In [ ]:
self.s.simulated_boardings_df.head()